In [1]:
import os
import numpy as np
import pandas as pd

import datetime as dt
from datetime import datetime, timedelta, time
from pandas import DataFrame

In [2]:
#데이터 불러오기
path = "/root/yerim/"
os.chdir(path)


train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')
log_train = pd.read_csv('data/log_train.csv')
log_test = pd.read_csv('data/log_test.csv')

In [3]:
#날짜형 변환
train['insert_time']=pd.to_datetime(train['insert_time'],format="%Y-%m-%d %H:%M:%S")
test['insert_time']=pd.to_datetime(test['insert_time'],format="%Y-%m-%d %H:%M:%S")

log_train['timestamp']=pd.to_datetime(log_train['timestamp'],format="%Y-%m-%d %H:%M:%S")
log_train['date_cd']=pd.to_datetime(log_train['date_cd'],format="%Y-%m-%d")

log_test['timestamp']=pd.to_datetime(log_test['timestamp'],format="%Y-%m-%d %H:%M:%S")
log_test['date_cd']=pd.to_datetime(log_test['date_cd'],format="%Y-%m-%d")

In [4]:
test.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,isjob
0,1255231,2022-06-01 00:12:24,29,265,19000000.0,11.4,NaN,694012,1963.0,0.0,...,0.0,기타,전월세,10000000.0,생활비,1.0,1.0,0.0,0.0,0
1,588236,2022-06-01 00:18:03,13,123,0.0,20.0,NaN,778437,1965.0,0.0,...,0.0,기타,기타가족소유,3000000.0,생활비,0.0,no_rehabilitation,6.0,17000000.0,0
2,859369,2022-06-01 00:18:10,38,113,1000000.0,19.9,NaN,808859,1988.0,0.0,...,0.0,기타,기타가족소유,5000000.0,생활비,0.0,no_rehabilitation,4.0,24000000.0,0
3,859369,2022-06-01 00:18:11,35,29,5000000.0,14.8,NaN,808859,1988.0,0.0,...,0.0,기타,기타가족소유,5000000.0,생활비,0.0,no_rehabilitation,4.0,24000000.0,0
4,859369,2022-06-01 00:18:11,1,61,9000000.0,12.4,NaN,808859,1988.0,0.0,...,0.0,기타,기타가족소유,5000000.0,생활비,0.0,no_rehabilitation,4.0,24000000.0,0


In [5]:
#사용할 컬럼만 선택. 하나의 application_id는 모두 같은 insert_time을 가짐
test_new = test[['application_id','user_id','insert_time']]

In [6]:
test_new = test_new.drop_duplicates(['application_id','user_id','insert_time'],keep="first")

In [7]:
test_new.shape #train 데이터에 있는 application_id의 수는 총 239,865개

(239865, 3)

In [8]:
test['application_id'].nunique()

239865

In [9]:
test_new['event'] = 'APPLY' #train에 기록된 각 applicaton_id의 insert_time을 'APPLY'한 시각이라고 정의

In [10]:
test_new.rename(columns = {'insert_time':'timestamp'},inplace=True) #이후 merge를 하기 위해 컬럼명 바꾸기

In [11]:
test_new

,application_id,user_id,timestamp,event
0,1255231,694012,2022-06-01 00:12:23,APPLY
1,588236,778437,2022-06-01 00:18:03,APPLY
2,859369,808859,2022-06-01 00:18:10,APPLY
5,1359099,604970,2022-06-01 00:22:23,APPLY
12,537736,739513,2022-06-01 00:26:23,APPLY
...,...,...,...,...
3255422,202360,174644,2022-06-29 12:59:32,APPLY
3255435,142390,288088,2022-06-29 14:26:49,APPLY
3255442,1413225,132008,2022-06-30 10:12:48,APPLY
3255460,629710,804933,2022-06-30 12:56:53,APPLY


In [12]:
log = pd.concat([log_train,log_test],axis=0) #3,4,5월의 기록을 6월에 사용하기 위해 두 데이터를 concat해서 사용

In [13]:
log = log.sort_values('timestamp')

In [14]:
log['mp_os'].value_counts(dropna=False)

Android    12331688
iOS         3312862
android     2195434
iPadOS         3029
NaN             980
Name: mp_os, dtype: int64

In [15]:
log[log['mp_os'].isna()]['event']

9269631    GetCreditInfo
8291457    GetCreditInfo
6386252    GetCreditInfo
9142356    GetCreditInfo
8300316    GetCreditInfo
               ...      
2586123    GetCreditInfo
2601218    GetCreditInfo
2615473    GetCreditInfo
2614902    GetCreditInfo
2618128    GetCreditInfo
Name: event, Length: 980, dtype: object

In [16]:
log.loc[log['mp_os'].isna(), 'event'] = 'Error'

In [17]:
log.loc[log['mp_os'].isna(), 'mp_os'] = 'unknown' 

In [18]:
log.loc[log['mp_os']=='android', 'event'] = 'Error2'

In [19]:
log = log.sort_values(['user_id','timestamp'])

In [20]:
log['timediff'] = log.groupby(['user_id','date_cd'])['timestamp'].diff(axis=0)
log['timediff_min'] = log['timediff'].dt.total_seconds()/60 #분으로 표현

In [22]:
def func(x) :
    if x > 30 :
        return "Session_End"
    else :
        return np.nan
    
log["Session"] = log["timediff_min"].apply(lambda x : func(x))

In [24]:
log['Session_point'] = np.where(log['Session']=='Session_End', 1,0)
log['Session_cum'] = log.groupby(['user_id','date_cd'])['Session_point'].cumsum()
log['Session_cum'] = log['Session_cum']+1

In [26]:
log = log.drop(['timediff','timediff_min','Session','Session_point'],axis=1)
log.rename(columns={'Session_cum':'Session'},inplace=True)

In [28]:
log = log[['user_id','event','timestamp','date_cd','mp_os','Session']]
log = log.sort_values(['user_id','timestamp'])

In [29]:
log['application_id']=np.nan #merge하기 위해 application_id컬럼 만들어주기

In [30]:
log = log[['application_id','user_id','timestamp','date_cd','event','mp_os','Session']]

In [31]:
log.head(3)

,application_id,user_id,timestamp,date_cd,event,mp_os,Session
8985228,NaN,1,2022-05-03 14:52:28,2022-05-03,Error2,android,1
8985230,NaN,1,2022-05-03 14:52:35,2022-05-03,Error2,android,1
437135,NaN,1,2022-06-16 23:58:41,2022-06-16,UseLoanManage,Android,1


In [32]:
log.drop(['date_cd'],axis=1,inplace=True)

In [33]:
test_new['mp_os']=np.nan #데이터 합치기 위해 mp_os칼럼 만들어주기
test_new['Session']=np.nan

In [34]:
test_new.head(3)

,application_id,user_id,timestamp,event,mp_os,Session
0,1255231,694012,2022-06-01 00:12:23,APPLY,NaN,NaN
1,588236,778437,2022-06-01 00:18:03,APPLY,NaN,NaN
2,859369,808859,2022-06-01 00:18:10,APPLY,NaN,NaN


In [35]:
concat_test = pd.concat([log,test_new],ignore_index=True)

In [36]:
concat_test = concat_test.sort_values(['user_id','timestamp'])

In [39]:
concat_test['date'] = concat_test['timestamp'].dt.date

In [40]:
concat_test.shape

(18083858, 7)

In [42]:
concat_test['Session'] = np.where(concat_test.groupby(['user_id','date'])['timestamp'].diff(axis=0).dt.total_seconds()/60 <30, concat_test['Session'].ffill(), concat_test['Session'])

In [43]:
concat_test['Session'].fillna(0,inplace=True)

In [44]:
concat_test2 = concat_test.copy()

In [45]:
concat_test2 = concat_test2.sort_values(['user_id','timestamp'])
concat_test2['application_id'] = concat_test2.groupby(['user_id','date','Session'])['application_id'].bfill()

In [46]:
concat_test2['application_id'].nunique()

239865

In [48]:
concat_test2['first_visit'] = concat_test2.sort_values(by=['user_id', 'date']).groupby('user_id').date.transform('first')
concat_test2['how_long']=concat_test2['date']-concat_test2['first_visit']

In [49]:
concat_test2['how_long'] = concat_test2['how_long'].dt.days
concat_test2['how_long'] = concat_test2['how_long']+1

In [50]:
concat_test2

,application_id,user_id,timestamp,event,mp_os,Session,date,first_visit,how_long
0,NaN,1,2022-05-03 14:52:28,Error2,android,1.0,2022-05-03,2022-05-03,1
1,NaN,1,2022-05-03 14:52:35,Error2,android,1.0,2022-05-03,2022-05-03,1
2,NaN,1,2022-06-16 23:58:41,UseLoanManage,Android,1.0,2022-06-16,2022-05-03,45
3,NaN,1,2022-06-16 23:58:41,Login,Android,1.0,2022-06-16,2022-05-03,45
4,NaN,1,2022-06-16 23:58:42,Error2,android,1.0,2022-06-16,2022-05-03,45
...,...,...,...,...,...,...,...,...,...
17843988,NaN,879696,2022-03-14 05:38:16,CompleteIDCertification,Android,1.0,2022-03-14,2022-03-14,1
17843989,NaN,879696,2022-03-14 05:42:14,EndLoanApply,Android,1.0,2022-03-14,2022-03-14,1
17843990,NaN,879698,2022-05-24 22:33:24,OpenApp,iOS,1.0,2022-05-24,2022-05-24,1
17843991,NaN,879698,2022-05-24 22:33:32,StartLoanApply,iOS,1.0,2022-05-24,2022-05-24,1


In [51]:
concat_test3 = concat_test2.copy()
concat_test3 = concat_test3.sort_values(['user_id','timestamp'])
concat_test3 = concat_test3[['user_id','date']]
concat_test3['visit']=1
concat_test3 = concat_test3.drop_duplicates(['user_id','date','visit'],keep="first")

In [52]:
concat_test3

,user_id,date,visit
0,1,2022-05-03,1
2,1,2022-06-16,1
5,7,2022-05-22,1
6,9,2022-05-21,1
9,11,2022-03-24,1
...,...,...,...
17843968,879693,2022-06-29,1
17843969,879694,2022-03-31,1
17843975,879695,2022-05-27,1
17843979,879696,2022-03-14,1


In [53]:
concat_test3['visit_sum'] = concat_test3.groupby('user_id')['visit'].cumsum()
concat_test3['visit_cumsum'] = concat_test3.groupby('user_id')['visit_sum'].shift(1) #전날까지 방문횟수이므로 shift(1)
concat_test3['visit_cumsum'].fillna(0,inplace=True)
concat_test3 = concat_test3.drop(['visit_sum'],axis=1)

In [54]:
concat_test3['from_last_visit']=concat_test3.groupby('user_id')['date'].diff(axis=0)
concat_test3['from_last_visit'] = concat_test3['from_last_visit'].dt.days
concat_test3['from_last_visit'].fillna(0,inplace=True)

In [55]:
concat_test3[concat_test3['user_id']==11]

,user_id,date,visit,visit_cumsum,from_last_visit
9,11,2022-03-24,1,0.0,0.0
21,11,2022-04-20,1,1.0,27.0
42,11,2022-06-09,1,2.0,50.0
47,11,2022-06-13,1,3.0,4.0


In [56]:
concat_test3['before_7days'] = np.where((concat_test3['from_last_visit']<=7)&(concat_test3['from_last_visit']!=0), 1, 0)
concat_test3['before_30days'] = np.where((concat_test3['from_last_visit']<=30)&(concat_test3['from_last_visit']!=0), 1, 0)

In [57]:
concat_test4 = pd.merge(concat_test2, concat_test3, how='left', on=['user_id','date'])

In [59]:
concat_test4['visit_rate'] = concat_test4['visit_cumsum'] / concat_test4['how_long']

In [60]:
concat_test4.drop(['first_visit'],axis=1,inplace=True)

In [63]:
concat_test4 = concat_test4.sort_values(['user_id','timestamp'])
concat_test4['mp_os']=concat_test4.groupby(['user_id','date'])['mp_os'].ffill()

In [64]:
concat_test4['mp_os'] = concat_test4.groupby('user_id')['mp_os'].ffill()

In [65]:
concat_test4.drop(['visit'],axis=1,inplace=True)

In [66]:
concat_test5 = concat_test4.copy()

In [67]:
concat_test5['last_session'] = concat_test5.sort_values(by=['user_id', 'timestamp']).groupby(['user_id','date']).Session.transform('last')

In [69]:
concat_test5 = concat_test5[['user_id','date','last_session']]
concat_test5 = concat_test5.drop_duplicates(['user_id','date','last_session'],keep="first")
concat_test5['last_session'] = np.where(concat_test5['last_session']==0, 1,concat_test5['last_session'])
concat_test5['session_cum']=concat_test5.groupby('user_id')['last_session'].cumsum()
concat_test5['session_sum'] = concat_test5.groupby('user_id')['session_cum'].shift(1) #이전의 총 session 수 이므로
concat_test5['session_sum'].fillna(0,inplace=True)

In [70]:
concat_test5[concat_test5['user_id']==11]

,user_id,date,last_session,session_cum,session_sum
9,11,2022-03-24,1.0,1.0,0.0
21,11,2022-04-20,1.0,2.0,1.0
42,11,2022-06-09,1.0,3.0,2.0
48,11,2022-06-13,1.0,4.0,3.0


In [71]:
concat_test5 = concat_test5.drop(['last_session','session_cum'],axis=1)

In [72]:
concat_test6 = pd.merge(concat_test4, concat_test5, how='left', on=['user_id','date'])

In [74]:
concat_test6['session_avg']=concat_test6['session_sum']/concat_test6['visit_cumsum']
concat_test6['session_avg'].fillna(0,inplace=True)

In [75]:
concat_test7 = concat_test6.copy()

In [76]:
concat_test7['from_last_visit_time'] = concat_test7.groupby('user_id')['timestamp'].diff(axis=0)

In [77]:
concat_test7['from_last_visit_time2'] = concat_test7.groupby(['user_id','application_id']).from_last_visit_time.head(1)
concat_test7 = concat_test7[concat_test7['application_id'].notna()]
concat_test7 = concat_test7[['application_id','user_id','from_last_visit_time2']]
concat_test7= concat_test7.drop_duplicates(['application_id','user_id'],keep="first")
concat_test7.rename(columns={'from_last_visit_time2':'from_last_visit_time'},inplace=True)

In [80]:
concat_test_fin = concat_test6[concat_test6['application_id'].notna()]

In [81]:
concat_test_fin['application_id'].nunique()

239865

In [83]:
concat_test_fin.shape

(1465072, 15)

In [84]:
concat_test_fin = concat_test_fin.drop_duplicates(['user_id','timestamp','event'],keep="first")

In [85]:
concat_test_fin.shape

(1453298, 15)

In [86]:
test_dummy = pd.get_dummies(concat_test_fin['event'], dummy_na=False)

In [88]:
concat_test_dummy = concat_test_fin.join(test_dummy.add_suffix('_1'))

In [90]:
concat_test_dummy.drop(['user_id','timestamp','event','mp_os','Session','date','how_long','visit_cumsum','from_last_visit',
                         'before_7days','before_30days','visit_rate','session_sum','session_avg'],axis=1,inplace=True)

In [91]:
test_dummy2 = concat_test_dummy.groupby('application_id').cumsum()

In [92]:
test_dummy2

,APPLY_1,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1
42,0,0,0,0,0,0,0,1,0,0,0,0,0,0
43,0,0,0,0,0,0,0,1,0,1,0,0,0,0
44,0,0,0,0,0,0,0,1,0,1,0,0,0,1
45,0,1,0,0,0,0,0,1,0,1,0,0,0,1
46,1,1,0,0,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18083828,0,0,0,0,0,0,0,1,0,0,0,0,0,0
18083829,0,0,0,0,0,0,0,1,0,1,0,0,0,0
18083830,0,0,0,0,0,0,0,1,0,1,0,0,0,1
18083831,0,1,0,0,0,0,0,1,0,1,0,0,0,1


In [93]:
test_dummy2['event_count_1']=test_dummy2.sum(axis=1)

In [94]:
concat_cumsum_test = pd.concat([concat_test_fin,test_dummy2],axis=1)

In [95]:
event_series = concat_test_fin.groupby('application_id').agg({'event':'unique'}).reset_index()

In [96]:
event_series_new = event_series.copy()

In [97]:
event_series_new.rename(columns={'event':'events_1'},inplace=True)

In [98]:
concat_test_fin2 = pd.merge(concat_cumsum_test, event_series_new, on="application_id")

In [99]:
concat_test_fin2.shape

(1453298, 31)

In [100]:
connect_time = pd.DataFrame(concat_test_fin2.groupby('application_id')['timestamp'].max() - concat_test_fin2.groupby('application_id')['timestamp'].min())
connect_time.reset_index(inplace=True)
connect_time.rename(columns={'timestamp':'connect_time'},inplace=True)

In [101]:
concat_test_fin2 = pd.merge(concat_test_fin2, connect_time, on='application_id')

In [102]:
concat_test_fin2['connect_time'].describe()

count                      1453298
mean     0 days 00:05:19.054851792
std      0 days 00:08:21.112246292
min                0 days 00:00:00
25%                0 days 00:01:38
50%                0 days 00:02:34
75%                0 days 00:05:00
max                0 days 10:38:42
Name: connect_time, dtype: object

In [104]:
df = concat_test_fin2.drop_duplicates(['user_id','application_id','event'],keep="first")

In [105]:
df['application_id'].nunique()

239865

In [106]:
test_dummy3 = pd.get_dummies(df['event'],dummy_na=False)

In [107]:
df2 = df.join(test_dummy3.add_suffix('_2'))

In [108]:
df2['event'].unique()

array(['OpenApp', 'StartLoanApply', 'ViewLoanApplyIntro',
       'CompleteIDCertification', 'APPLY', 'Login', 'Error2',
       'UseLoanManage', 'GetCreditInfo', 'EndLoanApply', 'SignUp',
       'Error', 'UseDSRCalc', 'UsePrepayCalc'], dtype=object)

In [109]:
test_dummy4 = df2[['application_id','APPLY_2','CompleteIDCertification_2','EndLoanApply_2', 'GetCreditInfo_2', 'Login_2', 
                    'OpenApp_2', 'SignUp_2', 'StartLoanApply_2','UseDSRCalc_2', 'UseLoanManage_2', 'UsePrepayCalc_2', 
                    'ViewLoanApplyIntro_2']].groupby('application_id').cumsum()

In [110]:
test_dummy4['event_count_2']=test_dummy4.sum(axis=1)

In [111]:
df3 = pd.concat([df, test_dummy4],axis=1)

In [112]:
df3['event'].value_counts()

APPLY                      239865
CompleteIDCertification    198574
ViewLoanApplyIntro         196092
StartLoanApply             170663
OpenApp                    162546
Login                      137598
Error2                      82577
UseLoanManage               43323
EndLoanApply                20249
GetCreditInfo               13623
SignUp                       1798
Error                          20
UsePrepayCalc                  16
UseDSRCalc                     11
Name: event, dtype: int64

In [113]:
error = df3[df3['event'] == 'Error'].index
df3.drop(error,axis='index',inplace=True)
error2 = df3[df3['event'] == 'Error2'].index
df3.drop(error2,axis='index',inplace=True)

In [114]:
event_series2 = df3.groupby('application_id').agg({'event':'unique'}).reset_index()
event_series2.rename(columns={'event':'events_2'},inplace=True)

In [115]:
event_series2

,application_id,events_2
0,4.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
1,8.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
2,9.0,"[OpenApp, Login, ViewLoanApplyIntro, StartLoan..."
3,35.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."
4,40.0,"[OpenApp, Login, ViewLoanApplyIntro, StartLoan..."
...,...,...
239860,2167767.0,"[OpenApp, ViewLoanApplyIntro, StartLoanApply, ..."
239861,2167774.0,[APPLY]
239862,2167778.0,"[EndLoanApply, ViewLoanApplyIntro, CompleteIDC..."
239863,2167791.0,"[OpenApp, Login, StartLoanApply, ViewLoanApply..."


In [116]:
test_fin3 = pd.merge(df3, event_series2, on="application_id")

In [117]:
test_fin3.columns

Index(['application_id', 'user_id', 'timestamp', 'event', 'mp_os', 'Session',
       'date', 'how_long', 'visit_cumsum', 'from_last_visit', 'before_7days',
       'before_30days', 'visit_rate', 'session_sum', 'session_avg', 'APPLY_1',
       'CompleteIDCertification_1', 'EndLoanApply_1', 'Error_1', 'Error2_1',
       'GetCreditInfo_1', 'Login_1', 'OpenApp_1', 'SignUp_1',
       'StartLoanApply_1', 'UseDSRCalc_1', 'UseLoanManage_1',
       'UsePrepayCalc_1', 'ViewLoanApplyIntro_1', 'event_count_1', 'events_1',
       'connect_time', 'APPLY_2', 'CompleteIDCertification_2',
       'EndLoanApply_2', 'GetCreditInfo_2', 'Login_2', 'OpenApp_2', 'SignUp_2',
       'StartLoanApply_2', 'UseDSRCalc_2', 'UseLoanManage_2',
       'UsePrepayCalc_2', 'ViewLoanApplyIntro_2', 'event_count_2', 'events_2'],
      dtype='object')

In [118]:
test_fin4 = test_fin3[test_fin3['event']=='APPLY']

In [119]:
test_fin4.shape

(239865, 46)

In [120]:
test_fin4.rename(columns={'timestamp':'insert_time'},inplace=True) 

/tmp/ipykernel_32425/3716699013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_fin4.rename(columns={'timestamp':'insert_time'},inplace=True)


In [121]:
test_fin4.head(3)

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,...,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2
4,1521731.0,11,2022-06-09 11:25:25,APPLY,iOS,1.0,2022-06-09,78,2.0,50.0,...,0,1,0,1,0,0,0,1,5,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ..."
9,1894024.0,11,2022-06-13 21:44:30,APPLY,iOS,1.0,2022-06-13,82,3.0,4.0,...,0,1,0,1,0,0,0,1,5,"[OpenApp, ViewLoanApplyIntro, StartLoanApply, ..."
16,972420.0,25,2022-06-11 10:49:17,APPLY,Android,1.0,2022-06-11,89,2.0,13.0,...,1,1,0,1,0,1,0,1,7,"[OpenApp, Login, UseLoanManage, StartLoanApply..."


In [122]:
test_fin4['mp_os'].value_counts()

Android    151639
iOS         51660
android     20465
iPadOS         99
unknown         2
Name: mp_os, dtype: int64

In [123]:
test_fin4[test_fin4['mp_os']=='unknown']

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,...,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2
667787,85864.0,494336,2022-06-21 11:18:23,APPLY,unknown,1.0,2022-06-21,1,0.0,0.0,...,0,0,0,0,0,0,0,0,1,[APPLY]
867618,1749335.0,642533,2022-06-03 11:34:09,APPLY,unknown,1.0,2022-06-03,1,0.0,0.0,...,0,0,0,0,0,0,0,0,1,[APPLY]


In [124]:
no_mp = test_fin4[test_fin4['mp_os'].isna()]['user_id'].values
no_mp = set(list(no_mp))

In [125]:
log_train.sort_values('timestamp',ascending=False)

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
6980793,543098,Login,2022-05-31 23:59:59,Android,3.14.0,2022-05-31
5522314,845575,ViewLoanApplyIntro,2022-05-31 23:59:57,Android,3.14.0,2022-05-31
5522312,845575,StartLoanApply,2022-05-31 23:59:57,Android,3.14.0,2022-05-31
3870180,115372,StartLoanApply,2022-05-31 23:59:54,Android,3.14.0,2022-05-31
3870181,115372,EndLoanApply,2022-05-31 23:59:54,Android,3.14.0,2022-05-31
...,...,...,...,...,...,...
2369589,706858,GetCreditInfo,2022-03-01 00:00:03,android,417,2022-03-01
5063346,706858,UseLoanManage,2022-03-01 00:00:03,Android,3.8.0,2022-03-01
432335,125129,UseLoanManage,2022-03-01 00:00:00,Android,3.8.0,2022-03-01
2922314,385811,Login,2022-03-01 00:00:00,Android,3.8.0,2022-03-01


In [126]:
mp_user = pd.DataFrame(log_train[log_train['user_id'].isin(no_mp)])

In [127]:
mp_user

,user_id,event,timestamp,mp_os,mp_app_version,date_cd


In [128]:
test_fin4.loc[test_fin4['mp_os'] == 'android', 'mp_os'] = 'Android' #android로 표기되어 있던 것 Android로 일치시킴
test_fin4.loc[test_fin4['mp_os'] == 'iPadOS', 'mp_os'] = 'iOS' #iPadOS로 표기되어 있던 것 iOS로 변경

In [129]:
test_fin4['mp_os'].value_counts(dropna=False)

Android    172104
iOS         51759
NaN         16000
unknown         2
Name: mp_os, dtype: int64

In [130]:
test_fin_real = test_fin4.copy()

In [131]:
test_fin_real['APPLY_cumsum_1'] = test_fin_real.groupby('user_id')['APPLY_1'].cumsum()
test_fin_real['CompleteIDCertification_cumsum_1'] = test_fin_real.groupby('user_id')['CompleteIDCertification_1'].cumsum()
test_fin_real['EndLoanApply_cumsum_1'] = test_fin_real.groupby('user_id')['EndLoanApply_1'].cumsum()
test_fin_real['GetCreditInfo_cumsum_1'] = test_fin_real.groupby('user_id')['GetCreditInfo_1'].cumsum()
test_fin_real['Login_cumsum_1'] = test_fin_real.groupby('user_id')['Login_1'].cumsum()
test_fin_real['OpenApp_cumsum_1'] = test_fin_real.groupby('user_id')['OpenApp_1'].cumsum()
test_fin_real['SignUp_cumsum_1'] = test_fin_real.groupby('user_id')['SignUp_1'].cumsum()
test_fin_real['StartLoanApply_cumsum_1'] = test_fin_real.groupby('user_id')['StartLoanApply_1'].cumsum()
test_fin_real['UseDSRCalc_cumsum_1'] = test_fin_real.groupby('user_id')['UseDSRCalc_1'].cumsum()
test_fin_real['UseLoanManage_cumsum_1'] = test_fin_real.groupby('user_id')['UseLoanManage_1'].cumsum()
test_fin_real['UsePrepayCalc_cumsum_1'] = test_fin_real.groupby('user_id')['UsePrepayCalc_1'].cumsum()
test_fin_real['ViewLoanApplyIntro_cumsum_1'] = test_fin_real.groupby('user_id')['ViewLoanApplyIntro_1'].cumsum()

In [132]:
test_fin_real['APPLY_cumsum_2'] = test_fin_real.groupby('user_id')['APPLY_2'].cumsum()
test_fin_real['CompleteIDCertification_cumsum_2'] = test_fin_real.groupby('user_id')['CompleteIDCertification_2'].cumsum()
test_fin_real['EndLoanApply_cumsum_2'] = test_fin_real.groupby('user_id')['EndLoanApply_2'].cumsum()
test_fin_real['GetCreditInfo_cumsum_2'] = test_fin_real.groupby('user_id')['GetCreditInfo_2'].cumsum()
test_fin_real['Login_cumsum_2'] = test_fin_real.groupby('user_id')['Login_2'].cumsum()
test_fin_real['OpenApp_cumsum_2'] = test_fin_real.groupby('user_id')['OpenApp_2'].cumsum()
test_fin_real['SignUp_cumsum_2'] = test_fin_real.groupby('user_id')['SignUp_2'].cumsum()
test_fin_real['StartLoanApply_cumsum_2'] = test_fin_real.groupby('user_id')['StartLoanApply_2'].cumsum()
test_fin_real['UseDSRCalc_cumsum_2'] = test_fin_real.groupby('user_id')['UseDSRCalc_2'].cumsum()
test_fin_real['UseLoanManage_cumsum_2'] = test_fin_real.groupby('user_id')['UseLoanManage_2'].cumsum()
test_fin_real['UsePrepayCalc_cumsum_2'] = test_fin_real.groupby('user_id')['UsePrepayCalc_2'].cumsum()
test_fin_real['ViewLoanApplyIntro_cumsum_2'] = test_fin_real.groupby('user_id')['ViewLoanApplyIntro_2'].cumsum()

In [133]:
test_fin_real.drop(['APPLY_1','APPLY_2','APPLY_cumsum_2'],axis=1,inplace=True)

In [134]:
test_fin_real['APPLY_cumsum_1'] = test_fin_real['APPLY_cumsum_1']-1

In [135]:
test_fin_real.rename(columns={'APPLY_cumsum_1':'before_apply_count'},inplace=True)

In [136]:
pd.set_option('display.max_columns', None) 

In [137]:
test_fin_real[test_fin_real['user_id']==11].sort_values('insert_time')

,application_id,user_id,insert_time,event,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2
4,1521731.0,11,2022-06-09 11:25:25,APPLY,iOS,1.0,2022-06-09,78,2.0,50.0,0,0,0.025641,2.0,1.0,1,0,0,0,0,0,1,0,1,0,0,0,1,5,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ...",0 days 00:02:07,1,0,0,0,1,0,1,0,0,0,1,5,"[OpenApp, StartLoanApply, ViewLoanApplyIntro, ...",0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1
9,1894024.0,11,2022-06-13 21:44:30,APPLY,iOS,1.0,2022-06-13,82,3.0,4.0,1,1,0.036585,3.0,1.0,1,0,0,0,0,0,1,0,1,0,0,0,1,5,"[OpenApp, ViewLoanApplyIntro, StartLoanApply, ...",0 days 00:01:29,1,0,0,0,1,0,1,0,0,0,1,5,"[OpenApp, ViewLoanApplyIntro, StartLoanApply, ...",1,2,0,0,0,2,0,2,0,0,0,2,2,0,0,0,2,0,2,0,0,0,2


In [138]:
test_fin_real.drop(['event'],axis=1,inplace=True)

In [139]:
test_fin_real2 = test_fin_real.copy()

In [140]:
test_fin_real2 = test_fin_real2.astype({'events_1':'str','events_2':'str'})

In [141]:
test_fin_real2["events_1"]= test_fin_real2["events_1"].str.replace(pat=r'[', repl=r'', regex=True)
test_fin_real2["events_1"]= test_fin_real2["events_1"].str.replace(pat=r']', repl=r'', regex=True)
test_fin_real2["events_1"]= test_fin_real2["events_1"].str.replace(pat=r"'", repl=r'', regex=True)
test_fin_real2["events_1"]= test_fin_real2["events_1"].str.replace(pat=r" ", repl=r'_', regex=True)
test_fin_real2["events_1"]= test_fin_real2["events_1"].str.replace(pat=r"\n", repl=r'', regex=True)

In [142]:
test_fin_real2["events_2"]= test_fin_real2["events_2"].str.replace(pat=r'[', repl=r'', regex=True)
test_fin_real2["events_2"]= test_fin_real2["events_2"].str.replace(pat=r']', repl=r'', regex=True)
test_fin_real2["events_2"]= test_fin_real2["events_2"].str.replace(pat=r"'", repl=r'', regex=True)
test_fin_real2["events_2"]= test_fin_real2["events_2"].str.replace(pat=r" ", repl=r'_', regex=True)
test_fin_real2["events_2"]= test_fin_real2["events_2"].str.replace(pat=r"\n", repl=r'', regex=True)

In [143]:
test_fin_real2.head()

,application_id,user_id,insert_time,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2
4,1521731.0,11,2022-06-09 11:25:25,iOS,1.0,2022-06-09,78,2.0,50.0,0,0,0.025641,2.0,1.000000,1,0,0,0,0,0,1,0,1,0,0,0,1,5,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,0 days 00:02:07,1,0,0,0,1,0,1,0,0,0,1,5,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1
9,1894024.0,11,2022-06-13 21:44:30,iOS,1.0,2022-06-13,82,3.0,4.0,1,1,0.036585,3.0,1.000000,1,0,0,0,0,0,1,0,1,0,0,0,1,5,OpenApp_ViewLoanApplyIntro_StartLoanApply_Comp...,0 days 00:01:29,1,0,0,0,1,0,1,0,0,0,1,5,OpenApp_ViewLoanApplyIntro_StartLoanApply_Comp...,1,2,0,0,0,2,0,2,0,0,0,2,2,0,0,0,2,0,2,0,0,0,2
16,972420.0,25,2022-06-11 10:49:17,Android,1.0,2022-06-11,89,2.0,13.0,0,1,0.022472,3.0,1.500000,1,0,0,2,0,1,1,0,1,0,1,0,1,9,OpenApp_Login_Error2_UseLoanManage_StartLoanAp...,0 days 00:01:57,1,0,0,1,1,0,1,0,1,0,1,7,OpenApp_Login_UseLoanManage_StartLoanApply_Vie...,0,1,0,0,1,1,0,1,0,1,0,1,1,0,0,1,1,0,1,0,1,0,1
22,1540551.0,27,2022-06-23 16:01:51,Android,1.0,2022-06-23,83,10.0,1.0,1,1,0.120482,17.0,1.700000,1,0,0,2,0,1,1,0,0,0,1,0,1,8,OpenApp_Login_UseLoanManage_Error2_ViewLoanApp...,0 days 00:04:09,1,0,0,1,1,0,0,0,1,0,1,6,OpenApp_Login_UseLoanManage_ViewLoanApplyIntro...,0,1,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,0,1,0,1
25,1669470.0,27,2022-06-29 18:05:53,Android,2.0,2022-06-29,89,15.0,1.0,1,1,0.168539,23.0,1.533333,1,0,0,0,0,0,0,0,0,0,0,0,1,3,ViewLoanApplyIntro_CompleteIDCertification_APPLY,0 days 00:01:16,1,0,0,0,0,0,0,0,0,0,1,3,ViewLoanApplyIntro_CompleteIDCertification_APPLY,1,2,0,0,1,1,0,0,0,1,0,2,2,0,0,1,1,0,0,0,1,0,2


In [144]:
test_fin_real2.shape

(239865, 66)

In [145]:
def event1(x) :
    if x == 'OpenApp_Login_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY' :
        return 1
    
    elif x == 'APPLY':
        return 2
    
    elif x == 'OpenApp_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 3
    
    elif x == 'Error2_APPLY':
        return 4
    
    elif x == 'OpenApp_Login_Error2_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 5
    
    else :
        return 6

test_fin_real2["event_group_1"] = test_fin_real2["events_1"].apply(lambda x : event1(x))

In [146]:
test_fin_real2["event_group_1"].value_counts()

6    140434
1     35171
2     23215
4     17821
3     12133
5     11091
Name: event_group_1, dtype: int64

In [147]:
def event2(x) :
    if x == 'OpenApp_Login_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY' :
        return 1
    
    elif x == 'APPLY':
        return 2
    
    elif x == 'OpenApp_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 3
    
    elif x == 'OpenApp_Login_UseLoanManage_StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 4
    
    elif x == 'StartLoanApply_ViewLoanApplyIntro_CompleteIDCertification_APPLY':
        return 5
    
    else :
        return 6

test_fin_real2["event_group_2"] = test_fin_real2["events_2"].apply(lambda x : event2(x))

In [148]:
test_fin_real2["event_group_2"].value_counts()

6    122833
1     50046
2     41038
3     12142
4      8705
5      5101
Name: event_group_2, dtype: int64

In [149]:
test_fin_real2.shape

(239865, 68)

In [150]:
test_fin_real3 = pd.merge(test_fin_real2, concat_test7, on=['application_id','user_id'])

In [151]:
test_fin_real3

,application_id,user_id,insert_time,mp_os,Session,date,how_long,visit_cumsum,from_last_visit,before_7days,before_30days,visit_rate,session_sum,session_avg,CompleteIDCertification_1,EndLoanApply_1,Error_1,Error2_1,GetCreditInfo_1,Login_1,OpenApp_1,SignUp_1,StartLoanApply_1,UseDSRCalc_1,UseLoanManage_1,UsePrepayCalc_1,ViewLoanApplyIntro_1,event_count_1,events_1,connect_time,CompleteIDCertification_2,EndLoanApply_2,GetCreditInfo_2,Login_2,OpenApp_2,SignUp_2,StartLoanApply_2,UseDSRCalc_2,UseLoanManage_2,UsePrepayCalc_2,ViewLoanApplyIntro_2,event_count_2,events_2,before_apply_count,CompleteIDCertification_cumsum_1,EndLoanApply_cumsum_1,GetCreditInfo_cumsum_1,Login_cumsum_1,OpenApp_cumsum_1,SignUp_cumsum_1,StartLoanApply_cumsum_1,UseDSRCalc_cumsum_1,UseLoanManage_cumsum_1,UsePrepayCalc_cumsum_1,ViewLoanApplyIntro_cumsum_1,CompleteIDCertification_cumsum_2,EndLoanApply_cumsum_2,GetCreditInfo_cumsum_2,Login_cumsum_2,OpenApp_cumsum_2,SignUp_cumsum_2,StartLoanApply_cumsum_2,UseDSRCalc_cumsum_2,UseLoanManage_cumsum_2,UsePrepayCalc_cumsum_2,ViewLoanApplyIntro_cumsum_2,event_group_1,event_group_2,from_last_visit_time
0,1521731.0,11,2022-06-09 11:25:25,iOS,1.0,2022-06-09,78,2.0,50.0,0,0,0.025641,2.0,1.000000,1,0,0,0,0,0,1,0,1,0,0,0,1,5,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,0 days 00:02:07,1,0,0,0,1,0,1,0,0,0,1,5,OpenApp_StartLoanApply_ViewLoanApplyIntro_Comp...,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,3,3,50 days 00:43:06
1,1894024.0,11,2022-06-13 21:44:30,iOS,1.0,2022-06-13,82,3.0,4.0,1,1,0.036585,3.0,1.000000,1,0,0,0,0,0,1,0,1,0,0,0,1,5,OpenApp_ViewLoanApplyIntro_StartLoanApply_Comp...,0 days 00:01:29,1,0,0,0,1,0,1,0,0,0,1,5,OpenApp_ViewLoanApplyIntro_StartLoanApply_Comp...,1,2,0,0,0,2,0,2,0,0,0,2,2,0,0,0,2,0,2,0,0,0,2,6,6,4 days 10:16:54
2,972420.0,25,2022-06-11 10:49:17,Android,1.0,2022-06-11,89,2.0,13.0,0,1,0.022472,3.0,1.500000,1,0,0,2,0,1,1,0,1,0,1,0,1,9,OpenApp_Login_Error2_UseLoanManage_StartLoanAp...,0 days 00:01:57,1,0,0,1,1,0,1,0,1,0,1,7,OpenApp_Login_UseLoanManage_StartLoanApply_Vie...,0,1,0,0,1,1,0,1,0,1,0,1,1,0,0,1,1,0,1,0,1,0,1,6,4,13 days 04:00:49
3,1540551.0,27,2022-06-23 16:01:51,Android,1.0,2022-06-23,83,10.0,1.0,1,1,0.120482,17.0,1.700000,1,0,0,2,0,1,1,0,0,0,1,0,1,8,OpenApp_Login_UseLoanManage_Error2_ViewLoanApp...,0 days 00:04:09,1,0,0,1,1,0,0,0,1,0,1,6,OpenApp_Login_UseLoanManage_ViewLoanApplyIntro...,0,1,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,0,1,0,1,6,6,0 days 16:52:28
4,1669470.0,27,2022-06-29 18:05:53,Android,2.0,2022-06-29,89,15.0,1.0,1,1,0.168539,23.0,1.533333,1,0,0,0,0,0,0,0,0,0,0,0,1,3,ViewLoanApplyIntro_CompleteIDCertification_APPLY,0 days 00:01:16,1,0,0,0,0,0,0,0,0,0,1,3,ViewLoanApplyIntro_CompleteIDCertification_APPLY,1,2,0,0,1,1,0,0,0,1,0,2,2,0,0,1,1,0,0,0,1,0,2,6,6,0 days 02:25:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239860,1885945.0,879683,2022-06-09 11:30:37,Android,1.0,2022-06-09,101,14.0,2.0,1,1,0.138614,17.0,1.214286,1,0,0,1,0,1,1,0,1,0,0,0,1,7,OpenApp_Login_Error2_ViewLoanApplyIntro_StartL...,0 days 00:03:08,1,0,0,1,1,0,1,0,0,0,1,6,OpenApp_Login_ViewLoanApplyIntro_StartLoanAppl...,0,1,0,0,1,1,0,1,0,0,0,1,1,0,0,1,1,0,1,0,0,0,1,6,6,2 days 01:09:12
239861,1345681.0,879683,2022-06-17 07:09:55,Android,1.0,2022-06-17,109,16.0,7.0,1,1,0.146789,19.0,1.187500,1,0,0,1,0,1,1,0,0,0,0,0,1,6,OpenApp_Login_Error2_ViewLoanApplyIntro_Comple...,0 days 00:03:37,1,0,0,1,1,0,0,0,0,0,1,5,OpenApp_Login_ViewLoanApplyIntro_CompleteIDCer...,1,2,0,0,2,2,0,1,0,0,0,2,2,0,0,2,2,0,1,0,0,0,2,6,6,6 days 19:05:50
239862,77601.0,879688,2022-06-20 21:11:18,NaN,0.0,2022-06-20,1,0.0,0.0,0,0,0.000000,0.0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0 days 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,APPLY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,NaT
239863,1229112.0,879693,2022-06-14 11:06:52,

In [152]:
test_fin_real3['from_last_visit_time'].fillna('0 days 00:00:00',inplace=True)

In [153]:
test_fin_real3.isna().sum()

application_id                     0
user_id                            0
insert_time                        0
mp_os                          16000
Session                            0
                               ...  
UsePrepayCalc_cumsum_2             0
ViewLoanApplyIntro_cumsum_2        0
event_group_1                      0
event_group_2                      0
from_last_visit_time               0
Length: 69, dtype: int64

In [154]:
test_fin_real3.shape

(239865, 69)

In [155]:
test_fin_real3.rename(columns={'event_count_1':'event_count1',
                               'event_count_2':'event_count2',
                               'event_group_1':'event_group1',
                               'event_group_2':'event_group2'},inplace=True)

In [156]:
test_fin_real3 = test_fin_real3[['application_id', 'user_id', 'insert_time', 'mp_os', 'Session', 'date', 'how_long', 
                                   'visit_cumsum', 'from_last_visit', 'visit_rate','session_sum', 'session_avg',
                                   'CompleteIDCertification_1','EndLoanApply_1', 'Error_1', 'Error2_1', 'GetCreditInfo_1',
                                   'Login_1','OpenApp_1', 'SignUp_1', 'StartLoanApply_1', 'UseDSRCalc_1','UseLoanManage_1', 
                                   'UsePrepayCalc_1', 'ViewLoanApplyIntro_1','events_1',
                                   'CompleteIDCertification_cumsum_1', 'EndLoanApply_cumsum_1','GetCreditInfo_cumsum_1', 
                                   'Login_cumsum_1', 'OpenApp_cumsum_1','SignUp_cumsum_1', 'StartLoanApply_cumsum_1',
                                   'UseDSRCalc_cumsum_1','UseLoanManage_cumsum_1','UsePrepayCalc_cumsum_1','ViewLoanApplyIntro_cumsum_1',
                                   'CompleteIDCertification_2','EndLoanApply_2', 'GetCreditInfo_2', 'Login_2',
                                   'OpenApp_2', 'SignUp_2', 'StartLoanApply_2', 'UseDSRCalc_2','UseLoanManage_2', 'UsePrepayCalc_2',
                                   'ViewLoanApplyIntro_2','events_2','CompleteIDCertification_cumsum_2','EndLoanApply_cumsum_2',
                                   'GetCreditInfo_cumsum_2', 'Login_cumsum_2','OpenApp_cumsum_2', 'SignUp_cumsum_2',
                                   'StartLoanApply_cumsum_2','UseDSRCalc_cumsum_2', 'UseLoanManage_cumsum_2','UsePrepayCalc_cumsum_2',
                                   'ViewLoanApplyIntro_cumsum_2','connect_time','from_last_visit_time','before_apply_count',
                                  'before_7days','before_30days','event_count1','event_count2','event_group1','event_group2']]

In [157]:
test_fin_real3.to_csv("log_test_fin(realreal).csv",index=False)